# Objective

1. Process the order data (zone-based,link-based,point-based, travel-time)


2. Initialize the driver data


3. Order dispatching


4. Repositioning location

In [1]:
import pandas as pd

from shapely.geometry import Point, Polygon

import geopandas as gp

import numpy as np

import random

import pulp

import folium

import networkx as nx


import math

In [1]:
def Check_zones(pnt,polys):
    
    key='None'
    
    for k, geom in polys.items():
        
        if pnt.within(geom):
            
            key=k
            
            break
            
    return key

def Check_links(pnt,zone,Zone_Link,Link_middle):
    
    Dic={}
    
    for link in Zone_Link[zone]:
        
        middle_pnt=Link_middle[link]
        
        Dic[link]=pnt.distance(middle_pnt)
        
    return min(Dic, key=Dic.get)

def Check_points(pnt,Zone_Points,Point_coordinate):
    
    Dic={}
    
    for point in Zone_Points:
        
        coord=Point_coordinate[point]
        
        Dic[point]=pnt.distance(coord)
        
    return min(Dic, key=Dic.get)
        

def Get_path(G,source,target,Point_coordinate):
    
    link_path=list()
    
    try:
    
        path=nx.shortest_path(G, source=source, target=target,weight='weight')

        shortest_dis=nx.shortest_path_length(G, source=source, target=target,weight='weight')
        
        link_path=path

        
    except:
        
        '''Path'''
        
        start_lng,start_lat=list(Point_coordinate[source].coords)[0][0],list(Point_coordinate[source].coords)[0][1]
        
        end_lng,end_lat=list(Point_coordinate[target].coords)[0][0],list(Point_coordinate[target].coords)[0][1]
        
        '''10 parts'''
        
        
        for i in range(1,10,1):
            
            pnt_lng=start_lng+(end_lng-start_lng)*(i/10)
            
            pnt_lat=start_lat+(end_lat-start_lat)*(i/10)
            
            pnt=Point(pnt_lng,pnt_lat)
            
            zone=Check_zones(pnt,polys)
            
            point=Check_points(pnt,zone,Point_coordinate)
            
            path_points.append(point)
               
        link_path=[source]+list(set(path_points))+[target]
            

        '''Distance'''
        
        shortest_dis=Point_coordinate[source].distance(Point_coordinate[target])*111000
    
        
    return link_path,shortest_dis


def Get_travel_time(dis,speed):
    
    return int(dis/speed)

def Get_matching_points(source,Point_coordinate,Points_list,radius):
    
    Point_candidates=list()
    
    for pnt in Points_list:
        
        dis=111000*(Point_coordinate[source].distance(Point_coordinate[pnt]))
    
        if dis<=radius:
            
            Point_candidates.append(pnt)
            
    return Point_candidates

def Compact_lists(arr):
    
    result=list()
    
    for a in arr:
        
        result=list(set(result+a))
        
    return result
            
def MILP_Optimization(Order,Driver,Utility):

    '''Define the problem'''

    model = pulp.LpProblem("Ride_Matching_Problems", pulp.LpMaximize)

    '''Construct our decision variable lists'''

    X = pulp.LpVariable.dicts("X",((i, j) for i in Order for j in Driver),lowBound=0,upBound=1,cat='Integer')

    '''Objective Function'''

    model += (pulp.lpSum([Utility[i][j] * X[(i, j)] for i in Order for j in Driver]))


    '''Each driver can only serve one order'''

    for i in Order:

        model += pulp.lpSum([X[(i, j)] for j in Driver]) <=1

    '''Each order can only be assigned one driver'''

    for j in Driver:

         model += pulp.lpSum([X[(i, j)] for i in Order]) <=1



    model.solve()

    result={}

    for var in X:

        var_value = X[var].varValue
        
        if var_value !=0:
            
            result[var[0]]=var[1]
    

    return result   
    
    

In [3]:
'''Param'''

s_sec=25200

e_sec=36000

Start_step=2520

End_step=3600

Max_waiting=12

radius=2000

speed=10 # 10 m/seconds

Driver_num=3000





In [4]:
'''Load data'''

'''Zone-related data'''

Taxi_Zones=np.load('./Data/NYC_Zones/Taxi_Zones.npy',allow_pickle=True).item()

Zone_list=np.load('./Data/NYC_Zones/Zone_list.npy',allow_pickle=True)

Zone_Center=np.load('./Data/NYC_Zones/Zone_Center.npy',allow_pickle=True).item()

Zone_Link=np.load('./Data/NYC_Zones/Zone_Link.npy',allow_pickle=True).item()

'''Link-related data'''

Link_list=np.load('./Data/NYC_Zones/Link_list.npy',allow_pickle=True)

Link_geometry=np.load('./Data/NYC_Zones/Link_geometry.npy',allow_pickle=True).item()


'''Point-related data'''

Points_list=np.load('./Data/NYC_Zones/Points_list.npy',allow_pickle=True)

Link_Point=np.load('./Data/NYC_Zones/Link_Point.npy',allow_pickle=True).item()

Point_coordinate=np.load('./Data/NYC_Zones/Point_coordinate.npy',allow_pickle=True).item()

Zone_Point=np.load('./Data/NYC_Zones/Zone_Point.npy',allow_pickle=True).item()

Point_zone=np.load('./Data/NYC_Zones/Point_zone.npy',allow_pickle=True).item()


'''GeoSeries Object'''

polys = gp.GeoSeries(Taxi_Zones)

'''Road network Object'''

G = nx.Graph()

G.add_nodes_from(Points_list)

G.add_weighted_edges_from(list(Link_Point.values()))


# Don't run this code frequently !

In [5]:
'''1. Processing the Order data'''

'''Link based and point based'''

Order_df=pd.read_csv('./Data/NewYork_New/pick_day_1.csv',header=None,names=['Pick_Second','Trip_Duration',\
                                                     'Pickup_Latitude','Pickup_Longitude',\
                                                     'Dropoff_Latitude','Dropoff_Longitude',\
                                                     'Trip_Distance','Euclidean','Manhattan'])

Order_df['Pickup_Zone']=Order_df.apply(lambda x:Check_zones(Point(x['Pickup_Longitude'],x['Pickup_Latitude']),polys),axis=1)

Order_df['Dropoff_Zone']=Order_df.apply(lambda x:Check_zones(Point(x['Dropoff_Longitude'],x['Dropoff_Latitude']),polys),axis=1)

Order_df=Order_df.loc[(Order_df['Pickup_Zone']!='None')&(Order_df['Dropoff_Zone']!='None')&(Order_df['Pick_Second']>=s_sec)&(Order_df['Pick_Second']<e_sec)]

Order_df=Order_df.sort_values(by=['Pick_Second'])

Order_df=Order_df.rename(columns={'Pick_Second':'Arrive_Second'})

Order_df=Order_df.reset_index(drop=True)

Order_df['Order_id']=['O'+str(i) for i in Order_df.index]

Order_df['Driver_id']='Waiting'

Order_df['Arrive_step']=Order_df.apply(lambda x:math.ceil(x['Arrive_Second']/10),axis=1)

Order_df=Order_df[['Order_id','Driver_id','Arrive_step',\
                   'Pickup_Latitude','Pickup_Longitude',\
                   'Dropoff_Latitude','Dropoff_Longitude',\
                    'Pickup_Zone','Dropoff_Zone']]

Order_df['Pickup_Point']=Order_df.apply(lambda x:Check_points(Point(x['Pickup_Longitude'],x['Pickup_Latitude']),\
                                                             Zone_Point[x['Pickup_Zone']],Point_coordinate),axis=1)

Order_df['Dropoff_Point']=Order_df.apply(lambda x:Check_points(Point(x['Dropoff_Longitude'],x['Dropoff_Latitude']),\
                                                             Zone_Point[x['Dropoff_Zone']],Point_coordinate),axis=1)

Order_df['Travel_dis']=Order_df.apply(lambda x:Get_path(G,x['Pickup_Point'],x['Dropoff_Point'],Point_coordinate)[1],axis=1)

Order_df['Travel_time']=Order_df.apply(lambda x:Get_travel_time(x['Travel_dis'],speed),axis=1)

# Order_df.to_csv('./Data/NYC_Trips/Order_df.csv')

Order_df



,Order_id,Driver_id,Arrive_step,Pickup_Latitude,Pickup_Longitude,Dropoff_Latitude,Dropoff_Longitude,Pickup_Zone,Dropoff_Zone,Pickup_Point,Dropoff_Point,Travel_dis,Travel_time
0,O0,Waiting,2520,40.748592,-73.987480,40.764927,-73.983566,Zone_43,Zone_42,Point_1769,Point_5325,6949.736391,694
1,O1,Waiting,2521,40.767056,-73.953827,40.755241,-73.975853,Zone_30,Zone_40,Point_2489,Point_2094,9952.610275,995
2,O2,Waiting,2521,40.760441,-73.991577,40.738243,-73.998856,Zone_8,Zone_16,Point_2682,Point_4944,9818.965513,981
3,O3,Waiting,2521,40.756618,-73.989731,40.769596,-73.954613,Zone_18,Zone_30,Point_1988,Point_501,14475.619478,1447
4,O4,Waiting,2521,40.725529,-74.005234,40.816872,-73.941338,Zone_17,Zone_6,Point_1492,Point_482,38782.910896,3878
5,O5,Waiting,2521,40.788265,-73.976349,40.779099,-73.944725,Zone_61,Zone_68,Point_3894,Point_5277,10842.361681,1084
6,O6,Waiting,2521,40.766899,-73.983192,40.767876,-73.993156,Zone_8,Zone_9,Point_2303,Point_4149,3761.352527,376
7,O7,Waiting,2521,40.743622,-73.984024,40.748802,-73.991043,Zone_43,Zone_46,Point_3586,Point_2724,3279.151179,327
8,O8,Waiting,2522,40.774574,-73.957504,40.778034,-73.974724,Zone_59,Zone_55,Point_6187,Point_3278,6889.752882,688
9,O9,Waiting,2522,40.746571,-74.001511,40.764679,-73.970932,Zone_10,Zone_60,Point_163,Point_5095,14138.641980,1413


In [6]:
'''2. Initialize the driver data'''

Driver_list=['D'+str(i) for i in range(Driver_num)]

Driver_df=pd.DataFrame(Driver_list,columns=['Driver_id'])

Driver_df['Point']=Driver_df.apply(lambda x:random.choice(Points_list),axis=1)

Driver_df['Zone']=Driver_df.apply(lambda x:Point_zone[x['Point']],axis=1)

Driver_df['Reposition_Point']=Driver_df.apply(lambda x:x['Point'],axis=1)

Driver_df['second']=s_sec

Driver_df['Step']=Driver_df.apply(lambda x:math.ceil(x['second']/10),axis=1)

Driver_df['Order_id']='Idle'

Driver_df=Driver_df[['Driver_id','Order_id','Step','Point','Zone','Reposition_Point']]

'''Initiial zones'''

# Driver_df.to_csv('./Data/NYC_Trips/Driver_df.csv')

Driver_df



,Driver_id,Order_id,Step,Point,Zone,Reposition_Point
0,D0,Idle,2520,Point_3032,Zone_13,Point_3032
1,D1,Idle,2520,Point_2993,Zone_12,Point_2993
2,D2,Idle,2520,Point_1788,Zone_44,Point_1788
3,D3,Idle,2520,Point_792,Zone_59,Point_792
4,D4,Idle,2520,Point_5285,Zone_47,Point_5285
5,D5,Idle,2520,Point_2518,Zone_6,Point_2518
6,D6,Idle,2520,Point_3419,Zone_47,Point_3419
7,D7,Idle,2520,Point_5492,Zone_6,Point_5492
8,D8,Idle,2520,Point_6030,Zone_12,Point_6030
9,D9,Idle,2520,Point_4243,Zone_12,Point_4243


In [7]:
'''Dispatching Procedure'''

Order_df=pd.read_csv('./Data/NYC_Trips/Order_df.csv')

Order_df=Order_df.drop(columns=['Unnamed: 0'])

Order_df['Response_step']=End_step

Order_df['Pickup_step']=End_step

Order_df=Order_df[['Order_id',\
                   'Driver_id',
                   'Arrive_step',\
                   'Response_step',\
                   'Pickup_step',\
                   'Pickup_Latitude',\
                   'Pickup_Longitude',\
                   'Dropoff_Latitude',\
                   'Dropoff_Longitude',\
                   'Pickup_Zone',\
                   'Dropoff_Zone',\
                   'Pickup_Point',\
                   'Dropoff_Point',\
                   'Travel_dis',\
                   'Travel_time']]

Driver_df=pd.read_csv('./Data/NYC_Trips/Driver_df.csv')

Driver_df=Driver_df.drop(columns=['Unnamed: 0'])


In [8]:
Quatitive_results=list()


for step in range(Start_step,End_step,1):
    
    
    '''(1) select unserved orders and arriving orders'''
    
    Order_batch=Order_df[(Order_df['Arrive_step']<=step)&(Order_df['Driver_id']=='Waiting')]
    
    Order_info={}
    
    for idx,row in Order_batch.iterrows():
        
        
        Order_info[row['Order_id']]={'Pickup_Point':row['Pickup_Point'],\
                              'Dropoff_Point':row['Dropoff_Point'],\
                              'Travel_time':int(row['Travel_time']/10),\
                              'Match_Points':Get_matching_points(row['Pickup_Point'],Point_coordinate,Points_list,radius)}
    
    Operation_Points=Compact_lists([x['Match_Points'] for x in Order_info.values()])
    

    '''(2) select Idle drivers'''
    
    Idle_drivers=list(Driver_df.loc[(Driver_df['Step']==step)&(Driver_df['Order_id']=='Idle'),'Driver_id'])
    
    Driver_batch=Driver_df[(Driver_df['Step']==step)&(Driver_df['Order_id']=='Idle')&(Driver_df['Point'].isin(Operation_Points))]
    
    Driver_info={}
    
    Points_Drivers={}
    
    for idx,row in Driver_batch.iterrows():
        
        Driver_info[row['Driver_id']]={'Point':row['Point']}
        
        if row['Point'] in Points_Drivers.keys():
            
            Points_Drivers[row['Point']].append(row['Driver_id'])
            
        else:
            
            Points_Drivers[row['Point']]=[row['Driver_id']]
            
            
    '''(3) Construct Matching Utility'''
    
    Utility={}
    
    Pickup_time={}

    for order_id in Order_info.keys():

        Utility[order_id]={}
        
        Pickup_time[order_id]={}
        
        origin=Order_info[order_id]['Pickup_Point']
        
        match_points=Order_info[order_id]['Match_Points']
        
        for point in match_points:
            
            '''Existing driver?'''
            
            if point in Points_Drivers.keys():
                
                for driver_id in Points_Drivers[point]:
                    
                    pickup_dis= Get_path(G,origin,point,Point_coordinate)[1]
                    
                    Pickup_time[order_id][driver_id]=int(Get_travel_time(pickup_dis,speed)/10)
                    
                    if pickup_dis!=0.0:
                        
                        Utility[order_id][driver_id]=1/(pickup_dis)
                        
                    else:
                        
                        Utility[order_id][driver_id]=99.0
                        
        for driver_id in Driver_info.keys():

            if driver_id not in Utility[order_id].keys():

                Utility[order_id][driver_id]=0.0
                
                Pickup_time[order_id][driver_id]=0
                
    '''(4) Optimize matching results '''
    
    Matching_result=MILP_Optimization(list(Order_info.keys()),list(Driver_info.keys()),Utility)
    
    print('Current step ',step,'Waiting orders: ',len(Order_info),'Selected drivers: ',len(Driver_info),'Matching pairs: ',len(Matching_result))
    
    Quatitive_results.append([step,len(Order_info),len(Idle_drivers),len(Matching_result)])

    print('*'*50)
    
 
    
    
    '''(5) Update results of orders'''
    
    '''(5-1) Update the matched orders and drivers'''
    
    for order_id,driver_id in Matching_result.items():
        
        '''Matched order'''
        
        Order_df.loc[(Order_df['Arrive_step']<=step)&(Order_df['Order_id']==order_id),'Response_step']=step
        
        Order_df.loc[(Order_df['Arrive_step']<=step)&(Order_df['Order_id']==order_id),'Pickup_step']=step+Pickup_time[order_id][driver_id]
        
        Order_df.loc[(Order_df['Arrive_step']<=step)&(Order_df['Order_id']==order_id),'Driver_id']=driver_id
        
        '''Matched driver'''

        Driver_df.loc[(Driver_df['Step']==step)&(Driver_df['Driver_id']==driver_id)&(Driver_df['Point'].isin(Operation_Points)),'Order_id']=order_id
        
        Added_item={'Driver_id': driver_id,\
                    'Order_id':'Idle',\
                    'Step':step+Pickup_time[order_id][driver_id]+Order_info[order_id]['Travel_time'],\
                    'Point':Order_info[order_id]['Dropoff_Point'],\
                    'Zone': Point_zone[Order_info[order_id]['Dropoff_Point']],\
                    'Reposition_Point':Order_info[order_id]['Dropoff_Point']}
        
        Driver_df=Driver_df.append(Added_item, ignore_index=True)
        
    '''(5-2) Update the unmatched orders'''
    
    
    Unmatched_orders=[O for O in Order_info.keys() if O not in Matching_result.keys()]
    
    if len(Unmatched_orders)!=0:
        
        Order_df.loc[((step-Order_df['Arrive_step'])>Max_waiting)&(Order_df['Order_id'].isin(Unmatched_orders)),'Driver_id']='Canceled'
        
        
    '''(5-3) Update the unmatehed drivers: Repositioning'''
    
    Idle_drivers=list(Driver_df.loc[(Driver_df['Step']==step)&(Driver_df['Order_id']=='Idle'),'Driver_id'])
    
    Next_Driver_df=Driver_df.loc[(Driver_df['Step']==step)&(Driver_df['Driver_id'].isin(Idle_drivers))]
    
    '''Just waiting at the current location'''
    
    Next_Driver_df['Step']=Next_Driver_df.apply(lambda x:x['Step']+1,axis=1)
    
    Driver_df=pd.concat([Driver_df,Next_Driver_df],ignore_index=True)
    

Current step  2520 Waiting orders:  1 Selected drivers:  399 Matching pairs:  1
**************************************************


/Users/yueyang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:149: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Current step  2521 Waiting orders:  7 Selected drivers:  1492 Matching pairs:  7
**************************************************
Current step  2522 Waiting orders:  6 Selected drivers:  1574 Matching pairs:  6
**************************************************
Current step  2523 Waiting orders:  10 Selected drivers:  1744 Matching pairs:  10
**************************************************
Current step  2524 Waiting orders:  7 Selected drivers:  847 Matching pairs:  7
**************************************************
Current step  2525 Waiting orders:  8 Selected drivers:  1425 Matching pairs:  8
**************************************************
Current step  2526 Waiting orders:  10 Selected drivers:  1503 Matching pairs:  10
**************************************************
Current step  2527 Waiting orders:  6 Selected drivers:  1233 Matching pairs:  6
**************************************************
Current step  2528 Waiting orders:  9 Selected drivers:  1338 Matching pa

Current step  2584 Waiting orders:  4 Selected drivers:  612 Matching pairs:  4
**************************************************
Current step  2585 Waiting orders:  9 Selected drivers:  928 Matching pairs:  9
**************************************************
Current step  2586 Waiting orders:  10 Selected drivers:  1456 Matching pairs:  10
**************************************************
Current step  2587 Waiting orders:  15 Selected drivers:  1078 Matching pairs:  15
**************************************************
Current step  2588 Waiting orders:  5 Selected drivers:  1110 Matching pairs:  5
**************************************************
Current step  2589 Waiting orders:  8 Selected drivers:  1329 Matching pairs:  8
**************************************************
Current step  2590 Waiting orders:  14 Selected drivers:  1682 Matching pairs:  14
**************************************************
Current step  2591 Waiting orders:  7 Selected drivers:  1131 Matching p